In [1]:
import zipfile
import os
import pandas as pd

In [2]:
def concatenar_csvs(diretorio):
    frames = []

    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(diretorio, arquivo)
            
            df = pd.read_csv(caminho_arquivo, encoding='utf-8', delimiter=";", decimal=",")
            df = df.rename(columns={df.columns[0]: "data_base"})
            df['data_base'] = pd.to_datetime(df['data_base'], format="%Y-%m-%d")
            df['mes'] = df['data_base'].dt.month #cria uma nova coluna com mês
            df['ano'] = df['data_base'].dt.year
            df = df[df['mes'] == 12]
            df = df[['ano', 'cliente', 'uf', 'ocupacao', 'cnae_secao', 'ativo_problematico']]
            df.loc[df['ocupacao'] == '-', 'ocupacao'] = df['cnae_secao'] #Adiciona o valor da coluna cnae_secao quando a coluna ocupacao é "-"
            df = df.groupby(['ano','cliente','uf', 'ocupacao'])['ativo_problematico'].sum().reset_index()
            
            frames.append(df)

    df_concatenado = pd.concat(frames, ignore_index=True)

    return df_concatenado

In [3]:
anos = list(range(2012, 2024))
dataframes = []

for ano in anos:
    diretorio = f"planilha_{ano}"
    dataframe_ano = concatenar_csvs(diretorio)
    dataframes.append(dataframe_ano)

df_total = pd.concat(dataframes, ignore_index=False)

In [4]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8200 entries, 0 to 759
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ano                 8200 non-null   int32  
 1   cliente             8200 non-null   object 
 2   uf                  8200 non-null   object 
 3   ocupacao            8200 non-null   object 
 4   ativo_problematico  8200 non-null   float64
dtypes: float64(1), int32(1), object(3)
memory usage: 352.3+ KB


In [5]:
df_total.head(3)

,ano,cliente,uf,ocupacao,ativo_problematico
0,2012,PF,AC,PF - Aposentado/pensionista,11448387.97
1,2012,PF,AC,PF - Autônomo,8955205.34
2,2012,PF,AC,PF - Empregado de empresa privada,6786123.20


In [6]:
df_total.to_csv("cliente_uf_ocupacao_carteira_ativa.csv")

In [7]:
import json
import requests

In [8]:
url = "https://raw.githubusercontent.com/giuliano-oliveira/geodata-br-states/main/geojson/br_states.json" #Temos que dar os créditos

response = requests.get(url)
geojson_data = response.json()

In [ ]:
#geojson_data

In [9]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [11]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Análise de Carteira Ativa'),
    
    html.Div([
        html.Label('Selecione uma ocupação:'),
        dcc.Dropdown(
            id='ocupacao-dropdown',
            options=[{'label': i, 'value': i} for i in df_total['ocupacao'].unique()],
            value=df_total['ocupacao'].iloc[0]
        ),
    ]),
    
    dcc.Graph(id='choropleth-map')
])

@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('ocupacao-dropdown', 'value')]
)
def update_choropleth(ocupacao_value):
    
    filtered_df = df_total[df_total['ocupacao'] == ocupacao_value]
    
    fig = px.choropleth_mapbox(filtered_df, 
                               geojson=geojson_data, 
                               locations='uf', 
                               color='ativo_problematico',
                               color_continuous_scale="sunsetdark",
                               range_color=(0, max(filtered_df['ativo_problematico'])),
                               animation_frame='ano', 
                               mapbox_style="open-street-map",
                               zoom=3, 
                               center={"lat": -17.14, "lon": -57.33},
                               opacity=1,
                               labels={'ativo_problematico':'Carteira Ativa',
                                       'uf': 'Unidade da Federação do Brasil'},
                               featureidkey="properties.SIGLA")
    
    fig.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)